## Tune Prompt for Training Data
We need to generate the training data. This is going to be crucial because it ultimately shapes how the model responds to queries and feels when interacting with the user. Because it's important we'll try some different prompts to see what creates the best output for us.

## Environment Setup
This step uses the following libraries:
|Library|License|
|-|-|
| [python-dotenv](https://github.com/theskumar/python-dotenv) | BSD-3-Clause |
| [pandas](https://pandas.pydata.org/docs/getting_started/overview.html) | BSD-3-Clause |
| [pydantic](https://github.com/pydantic/pydantic) | MIT |
| [OpenAI](https://github.com/openai/openai-python) | Apache 2.0 |

In [1]:
import os, json, random
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
import pandas as pd
from IPython.display import display
from pydantic import BaseModel

In [2]:
load_dotenv()
OAI_KEY = os.getenv("PAT_API_KEY")
client = OpenAI(api_key=OAI_KEY)

DOCUMENT    = "FM5_0"
PDF_PATH    = Path("pdfs/raw/fm5-0.pdf")
BASE_MODEL  = Path("QuantFactory/Llama-3.2-1B-GGUF")
GGUF_FILE   = "Llama-3.2-1B.Q8_0.gguf"
CACHE_DIR   = "hf_cache"
DATA_DIR    = DOCUMENT / BASE_MODEL / "data"
MODEL_DIR   = DOCUMENT / BASE_MODEL / "lora"
CHUNKED_DATA = DATA_DIR / "chunked" / "chunked.jsonl"
QA_DATA      = DATA_DIR / "qa"      / "qa_pairs.jsonl"

N_PASSAGES    = 30
OPENAI_MODEL  = "gpt-4o-mini"
TEMPERATURE   = 0.4

Using OpenAI's structured API, we can leverage Pydantic classes to make sure our response adheres to a format. This is a simple question/answer format.

In [3]:
class QA(BaseModel):
    question:   str
    answer:     str

Define some possible prompts that we can use.

In [4]:
PROMPTS = {
    "v1":   """You are a data‑labeling assistant. You will be given a passage from a text that is prepended with context.
                TASK: Read the passage, then extract the context and create a question and answer from the passage.
                - Ask a factual question whose answer is present in the passage, 5‑25 words long.
                - Answer must be copied verbatim (case‑insensitive) from the passage, ≤50 words.
                - Do NOT include the passage in your output.
                - If you cannot find a suitable fact, output "FAIL" in the answer.
                """,

    "v2":   """You turn passages into Q/A.
                RULES — reply with a single question and it's corresponding answer with associated metadata:
                    - question: the question (5‑25 words)
                    - answer: the answer (≤ 50 words)

                If no fact fits, return "FAIL" in answer.

                Example 1
                INPUT:
                 A decisive point is a geographic place or critical factor...
                OUTPUT:
                {"question": "What is a decisive point?", "answer":" A geographic place or critical factor"}

                Example 2
                INPUT:
                Mission command is the Army's approach to command and control...
                OUTPUT:
                {"question": "What is mission command?", "answer": "the Army's approach to command and control"}
                """,

    "v3":   """Read the passage and emit exactly ONE question/answer pair with source information:
                              "question" – a factual question (5‑25 words)
                              "answer"   – verbatim answer text from the passage (≤ 50 words)
                        """
}

Load the chunks from the pdf and do a test to make sure the openAI client is set up properly.

In [5]:
passages = []
with CHUNKED_DATA.open() as f:
    for line in f:
        passages.append(json.loads(line)["text"])

random.seed(42)
sampled = random.sample(passages, N_PASSAGES)

In [6]:
resp = client.responses.parse(
                model=OPENAI_MODEL,
                input = [
                    {
                        "role": "system",
                        "content": PROMPTS['v2']
                    },
                    {
                        "role": "user",
                        "content": sampled[0]
                    }
                ],
                text_format=QA
)

In [7]:
print(resp.output_parsed)

question='What should be summarized in the Environmental Considerations section?' answer="The commander's scheme of environmental actions required to support the operation plan, including issues and actions for all operation phases."


Now gather all the responses for the different prompts.

In [8]:
results = []

for name, template in PROMPTS.items():
    for idx, passage in enumerate(sampled):
        try:
            resp = client.responses.parse(
                model=OPENAI_MODEL,
                input = [
                    {
                        "role": "system",
                        "content": template
                    },
                    {
                        "role": "user",
                        "content": passage
                    }
                ],
                text_format=QA
            )
            q, a = resp.output_parsed.question, resp.output_parsed.answer
        except Exception as e:
            q, a = "", ""
            print(e)
        print(f"\r{name}: {idx + 1:02d}/{N_PASSAGES}", end="")

        results.append({
            "prompt_version": name,
            "passage":        passage,
            "question":       q,
            "answer":         a
        })

v3: 30/30

Now I'll inspect the output to see which version I like best. I sort by the "passage" column so I can compare the different outputs for each prompt side-by-side.

In [9]:
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 10000

df = pd.DataFrame(results)
display(df)

,prompt_version,passage,question,answer
0,v1,"used to structure this narrative. Refer to Appendix 4 (Geospatial Engineering) to Annex G (Engineer) as required.\n- (5) (U) Environmental Considerations. Summarize the commander's scheme of environmental actions required to support the operation plan, operation order, or concept plan. Identify issues and actions that should be addressed during all phases of the operation. Refer to Appendix 5 (Environmental Considerations) to Annex G (Engineer) as required.\n- (6) (U) Engineer Reconnaissance. State the scheme of engineer reconnaissance by task and purpose for engineer tactical and technical reconnaissance including infrastructure reconnaissance requirements.\n- b. (U) Tasks to Subordinate Units. List engineering tasks to specific units that are not assigned in the base plan or order. List tasks specific to engineering and mobility, countermobility, and survivability assets only as necessary to ensure unity of effort. Specific and detailed task descriptions should be done in each respective appendix as applicable.\n- c. (U) Coordinating Instructions. List only instructions applicable to two or more subordinate units not covered in the base plan or order. Provide additional coordinating instructions for the following:\n- (1) (U) Identify and list the times or events when obstacle control measures become effective.\n- (2) (U) List supported unit information requirements focused on mobility, countermobility, and survivability that must be considered by subordinate engineer staff officers or that the supported unit requires. This includes engineer-related commander's critical information requirements and perhaps the requests for information that have already been submitted to higher.\n- (3) (U) Explain and describe the countermobility and survivability timelines.\n- 4. (U) Sustainment. Identify sustainment priorities for engineer key tasks and specify additional sustainment instructions as necessary, and, at a minimum, address engineer Class IV and V locations. Refer to Annex F (Sustainment) as required.\n\n[page number] [CLASSIFICATION]\n\nFigure E-7. Sample Annex G (Engineer) format (continued)\n\n## [CLASSIFICATION]\n\n## ANNEX G (ENGINEER) TO OPERATION PLAN or ORDER [number] [(code name)]-[issuing headquarters] [(classification of title)]\n\n",What should be identified during all phases of the operation related to environmental considerations?,Identify issues and actions that should be addressed during all phases of the operation.
1,v1,"an approved contingency plan exists that closely resembles the emerging scenario, that plan can be refined or adapted as necessary and executed. Contingency plans are often phased, and they have specified end states. Contingency plans seek to re-establish conditions favorable to the United States. Contingency plans have an identified military objective and termination criteria. They address military operations ranging from humanitarian assistance to large-scale combat operations.\n- 2-33. Planning for a contingency encompasses the activities associated with the development of plans for the deployment, employment, sustainment, and redeployment of forces and resources in response to potential crises identified in joint strategic planning documents. The level of planning detail of contingency plans varies based on guidance and changes in the security environment. Planning details range from level 1 to level 4 as discussed in paragraphs 2-34 through 2-37. Although listed sequentially, during a crisis they may be conducted concurrently or compressed depending on the situation and conditions. (See JP 5-0 for more information on contingency planning and associated levels of planning detail.)\n\n## Level 1-Commander's Estimate\n\n- 2-34. The commander's estimate is the commander's initial assessment in which options are provided in a concise statement that defines who, what, when, where, why, and how the course of action will be implemented (JP 5-0). The com

Pick a version and get all of the training data before saving.

In [10]:
template = PROMPTS["v2"]

In [13]:
training_data = []
for idx, passage in enumerate(passages):
    try:
        resp = client.responses.parse(
            model=OPENAI_MODEL,
            input = [
                {
                    "role": "system",
                    "content": template
                },
                {
                    "role": "user",
                    "content": passage
                }
            ],
            text_format=QA
        )
        q, a = resp.output_parsed.question, resp.output_parsed.answer
    except Exception as e:
        q, a = "", ""
        print(e)
    print(f"\r{idx + 1:02d}/{len(passages)}", end="")

    training_data.append({
        "passage":        passage,
        "question":       q,
        "answer":         a
    })

462/462

In [14]:
with open(QA_DATA, "w", encoding="utf-8") as f:
    for d in training_data:
        f.write(json.dumps(d, ensure_ascii=False) + "\n")